In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%file mrlogit.py
import math
from mrjob.job import MRJob

class MRLogit(MRJob):

    def configure_options(self):
        super(MRLogit, self).configure_options()
        self.add_passthrough_option('--weights', type='str')
    
    @staticmethod
    def y(x, w):
        # TODO: implement logit function
    
    def mapper(self, key, line):
        w = map(float, self.options.weights.split(','))
        values = map(float, line.split(','))
        t, x = values[0], values[1:]
        x = [1.] + x
        # TODO: return intermediate gradient values

    def reducer(self, key, values):
        # TODO: calc final gradient value

In [ ]:
import math
import random
import mrlogit

random.seed(123)
w = [random.random() for i in range(3)]
a = 5e-2

logit = mrlogit.MRLogit.y

def loss():
    with open('../data/logit.txt') as src:
        res, count = 0., 0.
        for line in src:
            values = map(float, line.split(','))
            t, x = values[0], values[1:]
            x = [1.] + x
            count += 1
            res += t * math.log(logit(x, w) + 1e-32) + (t - 1.) * math.log(1. - logit(x, w) + 1e-32)
        return res / count

for i in range(100):
    _w = ','.join(map(str, w))
    job = mrlogit.MRLogit(args=['../data/logit.txt', '--weights', _w])
    with job.make_runner() as runner:
        runner.run()
        # TODO: update weights
    print loss()